In [89]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [99]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

url = "https://www.france24.com/ar/%D8%B1%D9%8A%D8%A7%D8%B6%D8%A9/"
page = requests.get(url, headers=headers)
print(page)

<Response [200]>


In [100]:
# parser-lxml = Change html to Python friendly format
# Obtain page's information
soup = BeautifulSoup(page.text, 'lxml')
# soup

In [101]:
data=[]
# Find the main div
main_div = soup.find('div', attrs={'class': 't-content t-content--page-builder'})

# Check if the main div is found
if main_div:
    sub_divs = main_div.find_all('div', attrs={'class': 'article__title'})

    for sub_div in sub_divs:
        h3_tag = sub_div.find('h2')
        if h3_tag:
            data.append(h3_tag.text)

data.pop(len(data)-1)
# # data
mydata = pd.DataFrame(data,columns = ['Text'])
mydata.to_csv('output.csv', encoding='utf-8-sig', index=False)

In [102]:
url = "https://www.elsport.com/"
page = requests.get(url, headers=headers)
print(page)

<Response [200]>


In [103]:
soup = BeautifulSoup(page.text, 'lxml')
# soup

In [104]:
data = []
# print(soup.prettify())

# Find the main div
main_div = soup.find('ul', class_='listnews news-feed-container newsfeed-list')

# Check if the main div is found
if main_div:
    sub_divs = main_div.find_all('a', attrs={'class':'news-title'})

    for sub_div in sub_divs:
        h4_tag = sub_div.find('h4')
        if h4_tag:
            data.append(h4_tag.text)
else:
    print("Main div not found.")

existing_data = pd.read_csv('output.csv')

new_data = pd.DataFrame(data, columns=['Text'])


# Step 3: Append Data to Existing DataFrame
# combined_data = existing_data.append(new_data, ignore_index=True)
combined_df = pd.concat([existing_data, new_data], ignore_index=True)

# Step 4: Save Combined Data to the Same CSV File
combined_df.to_csv('output.csv', encoding='utf-8-sig', index=False)


In [105]:
url = "https://arabic.rt.com/sport/news/"
page = requests.get(url, headers=headers)
print(page)

<Response [200]>


In [106]:
soup = BeautifulSoup(page.text, 'lxml')
# soup

In [107]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd


In [108]:

# Initialize the WebDriver (make sure you have the appropriate driver installed)
driver = webdriver.Chrome()

# Load the initial page
url = "https://arabic.rt.com/sport/news/"
driver.get(url)

# Wait for the "Load More" button to be clickable (adjust timeout as needed)
wait = WebDriverWait(driver, 10)

# Close the cookie banner if present
cookie_banner = driver.find_element(By.CLASS_NAME, "cookies__banner-wrapper")
if cookie_banner.is_displayed():
    cookie_banner.click()

# Initialize a variable to store the data
data = []

# Set the maximum number of clicks
max_clicks = 25

for click_count in range(max_clicks):
    try:
        # Wait for the "Load More" button to be clickable
        load_more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "more-button")))
        
        # Scroll the "Load More" button into view
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        
        # Click the "Load More" button
        load_more_button.click()
        
        # Wait for some time to let the content load (you may need to adjust the time based on the page behavior)
        time.sleep(5)
        
        # Get the updated HTML after clicking the button
        updated_html = driver.page_source
        
        # Parse the updated HTML with BeautifulSoup
        soup = BeautifulSoup(updated_html, 'html.parser')
        
        # Extract the data as needed (adjust based on the actual structure of the page)
        main_div = soup.find('div', attrs={'class': 'list-news js-listing'})
        if main_div:
            sub_divs = main_div.find_all('div', attrs={'class': 'list-news_text'})
            for sub_div in sub_divs:
                h4_tag = sub_div.find('a')
                if h4_tag:
                    data.append(h4_tag.text)
    except Exception as e:
        # If the button is not found or clickable, exit the loop
        print(f"An error occurred: {e}")
        break

# Close the WebDriver
driver.quit()

# Print or use the data as needed
# data
existing_data = pd.read_csv('output.csv')

new_data = pd.DataFrame(data, columns=['Text'])


# Step 3: Append Data to Existing DataFrame
# combined_data = existing_data.append(new_data, ignore_index=True)
combined_df = pd.concat([existing_data, new_data], ignore_index=True)

# Step 4: Save Combined Data to the Same CSV File
combined_df.to_csv('output.csv', encoding='utf-8-sig', index=False)


In [109]:
data = pd.read_csv('output.csv')

data['Text'] = data['Text'].str.replace('\n', '')
df = pd.DataFrame(data)

keywords = ["تنس", "جوائز", "ليفربول", "أفضل لاعب", "رياضة", "تنس", "كرة السلة", "دوري أبطال", "رياضي", "مباراة", "المونديال", "كأس أمم", "ريال مدريد", "يفوز", "فوز"
            , "الملعب", "دوري", "الدوري", "كرة", "لاعب","بطولة","بطولة","البطولة","الخسارة","العالم","نهائي","الجزاء","اللعب","إحصاءات","الفوز","مونديال","مدرب","مباريات"
            ,"المتأهلين","الألعاب","صدارة","المجموعات","الجماهير","منشطات","للفوز","تعادل","نجم","الرياضية","الهدف","القدم","مجريات","القدم","نادي"," لقب","المدافع","منتخب"]



def calculate_score(text):
    # Calculate the keyword_score based on the presence of keywords in the text
    keyword_score = sum(word in text for word in keywords)
    
    # Additional scoring logic based on text characteristics (e.g., length, sentiment, etc.)
    length_score = len(text.split())  # Example: Score based on the number of words
    
    # Combine the scores with adjusted weights
    final_score = 0.8 * keyword_score + 0.1 * length_score
    
    # Round the final score to one decimal place
    rounded_score = round(final_score, 1)
    
    # Normalize the final score to be between 0 and 10
    normalized_score = min(max(rounded_score, 0), 10)
 
    
    return normalized_score

# Calculate scores for each text in the DataFrame
df['Score'] = df['Text'].apply(calculate_score)
df[['Text', 'Score']]
df.to_csv('output.csv', encoding='utf-8-sig', index=False)

In [119]:
import pandas as pd
from nltk.stem.isri import ISRIStemmer

# Download the NLTK resource if you haven't already
import nltk
nltk.download('isri')

file_path = 'output.csv'
df = pd.read_csv(file_path)

# Create an instance of the ISRIStemmer
stemmer = ISRIStemmer()

# Define a function to apply stemming to a text
def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

df['Text'] = df['Text'].apply(stem_text)

df[column_name]

[nltk_data] Error loading isri: Package 'isri' not found in index


0       دور بطل روبا: نتر ميلان-أتلتيكو درد ونابولي-بر...
1       كرة قدم: ئنف بري دور ترك في 19 كان اول بعد عرض...
2       دور بطل روبا: يتد ودع بطل خسر على لعب أمام بير...
3       شار اهل تحد جدة... سعد حضن نسخ عشر من بطل كأس ...
4       لمذ "فز" يجر وسم على صلح حكم في سبق لجز فضل لع...
                              ...                        
3565    اعل تحد ورب لكر قدم "يويفا"، انه عزم خصص 331 م...
3566    لم بعد نجم ارج ونل يسي شرك في مونديال 2026، قئ...
3567    عهد رئس ندي هلل، فهد بن فل، منح كفأ كبر اعب فر...
3568    أجل برا بير يونخ وضف اون برل التي كانت قرر يوم...
3569    دول غرد يدو ظهر صرف قئد ندي صر، كريستيانو رونا...
Name: Text, Length: 3570, dtype: object

In [120]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import string
from nltk.corpus import stopwords
import re 

In [121]:
# first we define a list of arabic and english punctiations that we want to get rid of in our text

punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words('arabic')

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [122]:
import nltk
import pandas as pd
data = pd.read_csv('output.csv')

def preprocessing (text):
    

    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    text = re.sub('[A-Za-z0-9]',' ',text)
    
    #normalize
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    #next creating a list of substrings
    text = ' '.join(word for word in text.split() if word not in stop_words)


    # tokenize the text
    tokens = nltk.word_tokenize(text)

    return tokens


data['Processed_Text'] = df['Text'].apply(preprocessing)
print(data['Processed_Text'])
# data.to_csv('output22.csv', encoding='utf-8-sig', index=False)


0       [دور, بطل, روبا, نتر, ميلاناتلتيكو, درد, ونابو...
1           [كره, قدم, ءنف, بري, دور, اول, عرض, حكم, عدء]
2       [دور, بطل, روبا, يتد, ودع, بطل, خسر, علي, لعب,...
3       [شار, اهل, تحد, جده, سعد, حضن, نسخ, بطل, كاس, ...
4       [لمذ, فز, يجر, وسم, علي, صلح, حكم, سبق, لجز, ف...
                              ...                        
3565    [اعل, تحد, ورب, لكر, قدم, يويفا, انه, عزم, خصص...
3566    [نجم, ارج, ونل, يسي, شرك, مونديال, قءل, انه, ر...
3567    [عهد, رءس, ندي, هلل, فهد, بن, فل, منح, كفا, كب...
3568    [اجل, برا, بير, يونخ, وضف, اون, برل, كانت, قرر...
3569    [دول, غرد, يدو, ظهر, صرف, قءد, ندي, صر, كريستي...
Name: Processed_Text, Length: 3570, dtype: object
